In [2]:
import pandas as pd
import numpy as np
import nbimporter
import feature_engineering
import glob
import os

In [66]:
spielplan = pd.read_csv(r"C:\Users\Isabell\EMSI\emsi_ml\Spielplan\WM_Brasilien_2014_Spielplan.csv", delimiter = ";")

print(spielplan["Uhrzeit (MESZ)"])

KeyError: 'Uhrzeit (MESZ)'

In [3]:
folder_path = r"Spielplan"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

spielplan_list = []

for file in csv_files:
    spielplan = pd.read_csv(file, delimiter = ";")
    print(file)
    print(spielplan["Uhrzeit (MESZ)"])
    print("test")
    spielplan["DateTime"] = spielplan["Datum"] + " " + spielplan["Uhrzeit (MESZ)"]
    try:
        spielplan["DateTime"] = pd.to_datetime(spielplan["DateTime"], format='%d. %b %y %H:%M')
    except:
        spielplan["DateTime"] = pd.to_datetime(spielplan["DateTime"], format='%d.%m.%Y %H:%M')

    if "EM" in file:
        spielplan["Liga"] = "EM"
    elif "WM" in file:
        spielplan["Liga"] = "WM"
    #spielplan["Datum"] = pd.to_datetime(spielplan["Datum"], format = "%d. %b %y")
    spielplan_list.append(spielplan)
print(spielplan_list)

Spielplan\EM_Europa_2021_Spielplan.csv
0     21:00
1     15:00
2     18:00
3     21:00
4     15:00
5     18:00
6     21:00
7     15:00
8     18:00
9     21:00
10    18:00
11    21:00
12    15:00
13    18:00
14    21:00
15    15:00
16    18:00
17    21:00
18    15:00
19    18:00
20    21:00
21    15:00
22    18:00
23    21:00
24    18:00
25    18:00
26    18:00
27    18:00
28    21:00
29    21:00
30    21:00
31    21:00
32    18:00
33    18:00
34    21:00
35    21:00
36    18:00
37    21:00
38    18:00
39    21:00
40    18:00
41    21:00
42    18:00
43    21:00
44    18:00
45    21:00
46    18:00
47    21:00
48    21:00
49    21:00
50    21:00
Name: Uhrzeit (MESZ), dtype: object
test
Spielplan\EM_Frankreich_2016_Spielplan.csv
0     21:00
1     15:00
2     18:00
3     21:00
4     15:00
5     18:00
6     21:00
7     15:00
8     18:00
9     21:00
10    18:00
11    21:00
12    15:00
13    18:00
14    21:00
15    15:00
16    18:00
17    21:00
18    15:00
19    18:00
20    21:00
21    15:00
2